In [ ]:
# Cell 1: Create Analytics Schema
%%sql
CREATE SCHEMA IF NOT EXISTS CRYPTO_DB.ANALYTICS_CRYPTO;

In [ ]:
# Cell 2: Create Daily Performance Metrics Table
%%sql
CREATE OR REPLACE TABLE CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_PERFORMANCE (
    crypto_symbol VARCHAR(10),
    date_day DATE,
    open_price FLOAT,
    high_price FLOAT,
    low_price FLOAT,
    close_price FLOAT,
    volume FLOAT,
    daily_return FLOAT,
    daily_return_pct FLOAT,
    daily_volatility FLOAT,
    moving_avg_7d FLOAT,
    moving_avg_30d FLOAT,
    rsi_14d FLOAT,
    volume_change_pct FLOAT,
    PRIMARY KEY (crypto_symbol, date_day)
);

In [ ]:
# Cell 3: Create Weekly Performance Metrics Table
%%sql
CREATE OR REPLACE TABLE CRYPTO_DB.ANALYTICS_CRYPTO.WEEKLY_PERFORMANCE (
    crypto_symbol VARCHAR(10),
    week_start_date DATE,
    week_end_date DATE,
    open_price FLOAT,
    high_price FLOAT,
    low_price FLOAT,
    close_price FLOAT,
    avg_volume FLOAT,
    weekly_return FLOAT,
    weekly_return_pct FLOAT,
    weekly_volatility FLOAT,
    avg_rsi FLOAT,
    PRIMARY KEY (crypto_symbol, week_start_date)
);

In [ ]:
# Cell 4: Create Monthly Performance Metrics Table
%%sql
CREATE OR REPLACE TABLE CRYPTO_DB.ANALYTICS_CRYPTO.MONTHLY_PERFORMANCE (
    crypto_symbol VARCHAR(10),
    month_start_date DATE,
    month_end_date DATE,
    open_price FLOAT,
    high_price FLOAT,
    low_price FLOAT,
    close_price FLOAT,
    avg_volume FLOAT,
    monthly_return FLOAT,
    monthly_return_pct FLOAT,
    monthly_volatility FLOAT,
    max_daily_gain FLOAT,
    max_daily_loss FLOAT,
    PRIMARY KEY (crypto_symbol, month_start_date)
);

In [ ]:
# Cell 5: Create RSI Calculation Function
%%sql
CREATE OR REPLACE FUNCTION CRYPTO_DB.ANALYTICS_CRYPTO.CALCULATE_RSI(gains ARRAY, losses ARRAY)
RETURNS FLOAT
LANGUAGE PYTHON
RUNTIME_VERSION = '3.8'
HANDLER = 'calculate_rsi'
AS
$$
import numpy as np

def calculate_rsi(gains, losses):
    if not gains or not losses or len(gains) < 14 or len(losses) < 14:
        return None
    
    # Calculate average gain and average loss
    avg_gain = np.mean(gains[-14:])
    avg_loss = np.mean(losses[-14:])
    
    if avg_loss == 0:
        return 100.0
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return float(rsi)
$$;

In [ ]:
# Cell 6: Create Stored Procedure for Incremental Updates
%%sql
CREATE OR REPLACE PROCEDURE CRYPTO_DB.ANALYTICS_CRYPTO.UPDATE_CRYPTO_ANALYTICS()
RETURNS STRING
LANGUAGE SQL
AS
$$
DECLARE
    last_update_date DATE;
    current_date DATE;
BEGIN
    -- Get the last update date from the daily performance table
    SELECT COALESCE(MAX(date_day), '2020-01-01'::DATE) 
    INTO last_update_date 
    FROM CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_PERFORMANCE;
    
    -- Get current date
    SELECT CURRENT_DATE() INTO current_date;
    
    -- Update Daily Performance
    INSERT INTO CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_PERFORMANCE (
        crypto_symbol,
        date_day,
        open_price,
        high_price,
        low_price,
        close_price,
        volume,
        daily_return,
        daily_return_pct,
        daily_volatility,
        moving_avg_7d,
        moving_avg_30d,
        rsi_14d,
        volume_change_pct
    )
    WITH daily_data AS (
        SELECT 
            crypto_symbol,
            date_day,
            FIRST_VALUE(open) OVER (PARTITION BY crypto_symbol, date_day ORDER BY timestamp) AS open_price,
            MAX(high) AS high_price,
            MIN(low) AS low_price,
            LAST_VALUE(close) OVER (PARTITION BY crypto_symbol, date_day ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close_price,
            SUM(volume) AS volume
        FROM CRYPTO_DB.HARMONIZED_CRYPTO.CRYPTO_HARMONIZED
        WHERE date_day > last_update_date AND date_day <= current_date
        GROUP BY crypto_symbol, date_day, timestamp
    ),
    prior_day AS (
        SELECT 
            d.crypto_symbol,
            d.date_day,
            LAG(d.close_price) OVER (PARTITION BY d.crypto_symbol ORDER BY d.date_day) AS prev_close,
            LAG(d.volume) OVER (PARTITION BY d.crypto_symbol ORDER BY d.date_day) AS prev_volume
        FROM daily_data d
    ),
    price_changes AS (
        SELECT 
            d.crypto_symbol,
            d.date_day,
            d.open_price,
            d.high_price,
            d.low_price,
            d.close_price,
            d.volume,
            d.close_price - p.prev_close AS daily_return,
            CASE WHEN p.prev_close > 0 THEN (d.close_price - p.prev_close) / p.prev_close * 100 ELSE NULL END AS daily_return_pct,
            (d.high_price - d.low_price) / d.open_price * 100 AS daily_volatility,
            CASE WHEN p.prev_volume > 0 THEN (d.volume - p.prev_volume) / p.prev_volume * 100 ELSE NULL END AS volume_change_pct
        FROM daily_data d
        LEFT JOIN prior_day p ON d.crypto_symbol = p.crypto_symbol AND d.date_day = p.date_day
    ),
    moving_avgs AS (
        SELECT 
            crypto_symbol,
            date_day,
            AVG(close_price) OVER (PARTITION BY crypto_symbol ORDER BY date_day ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS moving_avg_7d,
            AVG(close_price) OVER (PARTITION BY crypto_symbol ORDER BY date_day ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS moving_avg_30d
        FROM price_changes
    ),
    -- For RSI calculation
    gains_losses AS (
        SELECT 
            crypto_symbol,
            date_day,
            CASE WHEN daily_return > 0 THEN daily_return ELSE 0 END AS gain,
            CASE WHEN daily_return < 0 THEN ABS(daily_return) ELSE 0 END AS loss
        FROM price_changes
    ),
    array_agg_data AS (
        SELECT 
            gl.crypto_symbol,
            gl.date_day,
            ARRAY_AGG(gl2.gain) OVER (PARTITION BY gl.crypto_symbol ORDER BY gl2.date_day ROWS BETWEEN 13 PRECEDING AND CURRENT ROW) AS gains_array,
            ARRAY_AGG(gl2.loss) OVER (PARTITION BY gl.crypto_symbol ORDER BY gl2.date_day ROWS BETWEEN 13 PRECEDING AND CURRENT ROW) AS losses_array
        FROM gains_losses gl
        JOIN gains_losses gl2 ON gl.crypto_symbol = gl2.crypto_symbol AND gl2.date_day <= gl.date_day
        QUALIFY ROW_NUMBER() OVER (PARTITION BY gl.crypto_symbol, gl.date_day ORDER BY gl2.date_day DESC) = 1
    )
    SELECT 
        pc.crypto_symbol,
        pc.date_day,
        pc.open_price,
        pc.high_price,
        pc.low_price,
        pc.close_price,
        pc.volume,
        pc.daily_return,
        pc.daily_return_pct,
        pc.daily_volatility,
        ma.moving_avg_7d,
        ma.moving_avg_30d,
        CRYPTO_DB.ANALYTICS_CRYPTO.CALCULATE_RSI(ad.gains_array, ad.losses_array) AS rsi_14d,
        pc.volume_change_pct
    FROM price_changes pc
    JOIN moving_avgs ma ON pc.crypto_symbol = ma.crypto_symbol AND pc.date_day = ma.date_day
    LEFT JOIN array_agg_data ad ON pc.crypto_symbol = ad.crypto_symbol AND pc.date_day = ad.date_day
    WHERE NOT EXISTS (
        SELECT 1 FROM CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_PERFORMANCE 
        WHERE crypto_symbol = pc.crypto_symbol AND date_day = pc.date_day
    )
    ORDER BY pc.crypto_symbol, pc.date_day;

    -- Update Weekly Performance
    -- First, delete any partial week data that may need to be recalculated
    DELETE FROM CRYPTO_DB.ANALYTICS_CRYPTO.WEEKLY_PERFORMANCE
    WHERE week_end_date >= DATE_TRUNC('WEEK', last_update_date);
    
    -- Insert new and updated weekly data
    INSERT INTO CRYPTO_DB.ANALYTICS_CRYPTO.WEEKLY_PERFORMANCE (
        crypto_symbol,
        week_start_date,
        week_end_date,
        open_price,
        high_price,
        low_price,
        close_price,
        avg_volume,
        weekly_return,
        weekly_return_pct,
        weekly_volatility,
        avg_rsi
    )
    WITH weekly_data AS (
        SELECT 
            crypto_symbol,
            DATE_TRUNC('WEEK', date_day) AS week_start_date,
            DATEADD('DAY', 6, DATE_TRUNC('WEEK', date_day)) AS week_end_date,
            FIRST_VALUE(open_price) OVER (PARTITION BY crypto_symbol, DATE_TRUNC('WEEK', date_day) ORDER BY date_day) AS open_price,
            MAX(high_price) AS high_price,
            MIN(low_price) AS low_price,
            LAST_VALUE(close_price) OVER (PARTITION BY crypto_symbol, DATE_TRUNC('WEEK', date_day) ORDER BY date_day ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close_price,
            AVG(volume) AS avg_volume,
            AVG(daily_volatility) AS weekly_volatility,
            AVG(rsi_14d) AS avg_rsi
        FROM CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_PERFORMANCE
        WHERE date_day > DATE_TRUNC('WEEK', last_update_date)
        AND date_day <= current_date
        GROUP BY crypto_symbol, DATE_TRUNC('WEEK', date_day), date_day
    )
    SELECT 
        crypto_symbol,
        week_start_date,
        week_end_date,
        open_price,
        high_price,
        low_price,
        close_price,
        avg_volume,
        close_price - open_price AS weekly_return,
        (close_price - open_price) / open_price * 100 AS weekly_return_pct,
        weekly_volatility,
        avg_rsi
    FROM weekly_data
    GROUP BY crypto_symbol, week_start_date, week_end_date, open_price, high_price, low_price, close_price, avg_volume, weekly_volatility, avg_rsi
    ORDER BY crypto_symbol, week_start_date;

    -- Update Monthly Performance (similar structure to weekly)
    DELETE FROM CRYPTO_DB.ANALYTICS_CRYPTO.MONTHLY_PERFORMANCE
    WHERE month_end_date >= DATE_TRUNC('MONTH', last_update_date);
    
    INSERT INTO CRYPTO_DB.ANALYTICS_CRYPTO.MONTHLY_PERFORMANCE (
        crypto_symbol,
        month_start_date,
        month_end_date,
        open_price,
        high_price,
        low_price,
        close_price,
        avg_volume,
        monthly_return,
        monthly_return_pct,
        monthly_volatility,
        max_daily_gain,
        max_daily_loss
    )
    WITH monthly_data AS (
        SELECT 
            crypto_symbol,
            DATE_TRUNC('MONTH', date_day) AS month_start_date,
            LAST_DAY(date_day) AS month_end_date,
            FIRST_VALUE(open_price) OVER (PARTITION BY crypto_symbol, DATE_TRUNC('MONTH', date_day) ORDER BY date_day) AS open_price,
            MAX(high_price) AS high_price,
            MIN(low_price) AS low_price,
            LAST_VALUE(close_price) OVER (PARTITION BY crypto_symbol, DATE_TRUNC('MONTH', date_day) ORDER BY date_day ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close_price,
            AVG(volume) AS avg_volume,
            AVG(daily_volatility) AS monthly_volatility,
            MAX(daily_return_pct) AS max_daily_gain,
            MIN(daily_return_pct) AS max_daily_loss
        FROM CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_PERFORMANCE
        WHERE date_day > DATE_TRUNC('MONTH', last_update_date)
        AND date_day <= current_date
        GROUP BY crypto_symbol, DATE_TRUNC('MONTH', date_day), date_day
    )
    SELECT 
        crypto_symbol,
        month_start_date,
        month_end_date,
        open_price,
        high_price,
        low_price,
        close_price,
        avg_volume,
        close_price - open_price AS monthly_return,
        (close_price - open_price) / open_price * 100 AS monthly_return_pct,
        monthly_volatility,
        max_daily_gain,
        max_daily_loss
    FROM monthly_data
    GROUP BY crypto_symbol, month_start_date, month_end_date, open_price, high_price, low_price, close_price, avg_volume, monthly_volatility, max_daily_gain, max_daily_loss
    ORDER BY crypto_symbol, month_start_date;

    RETURN 'Successfully updated analytics tables with data from ' || last_update_date || ' to ' || current_date;
END;
$$;


In [ ]:
# Cell 7: Create task to run the analytics update procedure daily
%%sql
CREATE OR REPLACE TASK CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_ANALYTICS_UPDATE_TASK
    WAREHOUSE = CRYPTO_WH
    SCHEDULE = 'USING CRON 0 1 * * * UTC'
AS
    CALL CRYPTO_DB.ANALYTICS_CRYPTO.UPDATE_CRYPTO_ANALYTICS();


In [ ]:
# Cell 8: Execute procedure manually to populate initial data
session.sql("CALL CRYPTO_DB.ANALYTICS_CRYPTO.UPDATE_CRYPTO_ANALYTICS()").collect()
print("Initial analytics data populated successfully.")

#

In [ ]:
# Cell 9: Resume the task for automated updates
%%sql
ALTER TASK CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_ANALYTICS_UPDATE_TASK RESUME;



In [ ]:
# Cell 10: View sample of daily performance data
daily_performance = session.sql("""
SELECT * FROM CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_PERFORMANCE
ORDER BY crypto_symbol, date_day DESC
LIMIT 10
""")
daily_performance.show()

In [ ]:
# Cell 11: View sample of weekly performance data
weekly_performance = session.sql("""
SELECT * FROM CRYPTO_DB.ANALYTICS_CRYPTO.WEEKLY_PERFORMANCE
ORDER BY crypto_symbol, week_start_date DESC
LIMIT 5
""")
weekly_performance.show()


In [ ]:
# Cell 12: Create a final summary view
%%sql
CREATE OR REPLACE VIEW CRYPTO_DB.ANALYTICS_CRYPTO.PERFORMANCE_SUMMARY AS
WITH latest_daily AS (
    SELECT 
        crypto_symbol,
        MAX(date_day) as latest_date
    FROM CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_PERFORMANCE
    GROUP BY crypto_symbol
),
latest_values AS (
    SELECT 
        d.crypto_symbol,
        d.date_day,
        d.close_price as current_price,
        d.daily_return_pct as daily_change_pct,
        d.rsi_14d,
        d.moving_avg_30d
    FROM CRYPTO_DB.ANALYTICS_CRYPTO.DAILY_PERFORMANCE d
    JOIN latest_daily ld ON d.crypto_symbol = ld.crypto_symbol AND d.date_day = ld.latest_date
),
weekly_metrics AS (
    SELECT
        crypto_symbol,
        MAX(week_start_date) as latest_week,
        weekly_return_pct
    FROM CRYPTO_DB.ANALYTICS_CRYPTO.WEEKLY_PERFORMANCE
    QUALIFY ROW_NUMBER() OVER (PARTITION BY crypto_symbol ORDER BY week_start_date DESC) = 1
),
monthly_metrics AS (
    SELECT
        crypto_symbol,
        MAX(month_start_date) as latest_month,
        monthly_return_pct,
        max_daily_gain,
        max_daily_loss
    FROM CRYPTO_DB.ANALYTICS_CRYPTO.MONTHLY_PERFORMANCE
    QUALIFY ROW_NUMBER() OVER (PARTITION BY crypto_symbol ORDER BY month_start_date DESC) = 1
)
SELECT
    lv.crypto_symbol,
    lv.date_day as last_updated,
    lv.current_price,
    lv.daily_change_pct,
    wm.weekly_return_pct,
    mm.monthly_return_pct,
    lv.rsi_14d,
    CASE 
        WHEN lv.rsi_14d > 70 THEN 'Overbought'
        WHEN lv.rsi_14d < 30 THEN 'Oversold'
        ELSE 'Neutral'
    END as rsi_signal,
    CASE
        WHEN lv.close_price > lv.moving_avg_30d THEN 'Above 30D MA'
        ELSE 'Below 30D MA'
    END as trend_signal,
    mm.max_daily_gain,
    mm.max_daily_loss
FROM latest_values lv
JOIN weekly_metrics wm ON lv.crypto_symbol = wm.crypto_symbol
JOIN monthly_metrics mm ON lv.crypto_symbol = mm.crypto_symbol
ORDER BY lv.crypto_symbol;